In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
import gc

def prepare_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    x_train_2d = x_train.reshape(x_train.shape[0], -1) / 255.0
    x_test_2d = x_test.reshape(x_test.shape[0], -1) / 255.0

    x_train_3d = x_train.reshape(x_train.shape[0], 28, 28, 1) / 255.0
    x_test_3d = x_test.reshape(x_test.shape[0], 28, 28, 1) / 255.0

    x_train_tl = tf.image.resize(x_train_3d, [48, 48])
    x_train_tl = tf.image.grayscale_to_rgb(x_train_tl)
    x_test_tl = tf.image.resize(x_test_3d, [48, 48])
    x_test_tl = tf.image.grayscale_to_rgb(x_test_tl)

    return (x_train_2d, x_test_2d), (x_train_3d, x_test_3d), (x_train_tl, x_test_tl), (y_train, y_test)

def train_knn(x_train, x_test, y_train, y_test):
    print("\nTraining KNN model...")

    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
    knn.fit(x_train_scaled, y_train)

    y_pred = knn.predict(x_test_scaled)
    print("KNN Accuracy:", accuracy_score(y_test, y_pred))
    print("\nKNN Classification Report:")
    print(classification_report(y_test, y_pred))

    del x_train_scaled, x_test_scaled
    gc.collect()

    return knn

def create_mlp():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(784,)),
        Dropout(0.2),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

def create_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

def create_transfer_learning_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

def train_neural_network(model, x_train, x_test, y_train, y_test, model_name, learning_rate=0.001, batch_size=32, epochs=10):
    print(f"\nTraining {model_name}...")

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        verbose=1
    )

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"\n{model_name} Test Accuracy:", test_acc)

    tf.keras.backend.clear_session()
    gc.collect()

    return model, history

def main():
    (x_train_2d, x_test_2d), (x_train_3d, x_test_3d), (x_train_tl, x_test_tl), (y_train, y_test) = prepare_data()

    knn_model = train_knn(x_train_2d, x_test_2d, y_train, y_test)

    mlp_model = create_mlp()
    mlp_model, mlp_history = train_neural_network(
        mlp_model, x_train_2d, x_test_2d, y_train, y_test,
        "MLP", learning_rate=0.001, batch_size=32, epochs=10
    )

    cnn_model = create_cnn()
    cnn_model, cnn_history = train_neural_network(
        cnn_model, x_train_3d, x_test_3d, y_train, y_test,
        "CNN", learning_rate=0.001, batch_size=32, epochs=10
    )

    tl_model = create_transfer_learning_model()
    tl_model, tl_history = train_neural_network(
        tl_model, x_train_tl, x_test_tl, y_train, y_test,
        "Transfer Learning CNN", learning_rate=0.0001, batch_size=32, epochs=10
    )

if __name__ == "__main__":
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

    main()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Training KNN model...
KNN Accuracy: 0.9443

KNN Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       980
           1       0.96      0.99      0.97      1135
           2       0.96      0.93      0.94      1032
           3       0.92      0.95      0.94      1010
           4       0.94      0.94      0.94       982
           5       0.93      0.92      0.93       892
           6       0.96      0.97      0.97       958
           7       0.94      0.92      0.93      1028
           8       0.96      0.90      0.93       974
           9       0.92      0.92      0.92      1009

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training MLP...
Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.8682 - loss: 0.4253 - val_accuracy: 0.9647 - val_loss: 0.1194
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.9631 - loss: 0.1254 - val_accuracy: 0.9715 - val_loss: 0.0937
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 24s 14ms/step - accuracy: 0.9722 - loss: 0.0912 - val_accuracy: 0.9682 - val_loss: 0.1100
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - accuracy: 0.9770 - loss: 0.0763 - val_accuracy: 0.9752 - val_loss: 0.0862
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 36s 12ms/step - accuracy: 0.9800 - loss: 0.0651 - val_accuracy: 0.9731 - val_loss: 0.0953
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.9819 - loss: 0.0569 - val_accuracy: 0.9782 - val_loss: 0.0835
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 23s 13ms/step - accuracy: 0.9857 - loss: 0.0466 - val_accuracy: 0.9762 - val_loss: 0.0924
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - a

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training CNN...
Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 57s 37ms/step - accuracy: 0.8833 - loss: 0.3759 - val_accuracy: 0.9823 - val_loss: 0.0610
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 57s 38ms/step - accuracy: 0.9836 - loss: 0.0518 - val_accuracy: 0.9841 - val_loss: 0.0613
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 77s 35ms/step - accuracy: 0.9889 - loss: 0.0354 - val_accuracy: 0.9878 - val_loss: 0.0425
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 83s 36ms/step - accuracy: 0.9924 - loss: 0.0250 - val_accuracy: 0.9866 - val_loss: 0.0493
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 55s 37ms/step - accuracy: 0.9928 - loss: 0.0211 - val_accuracy: 0.9903 - val_loss: 0.0358
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 83s 38ms/step - accuracy: 0.9959 - loss: 0.0133 - val_accuracy: 0.9908 - val_loss: 0.0382
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 81s 37ms/step - accuracy: 0.9958 - loss: 0.0125 - val_accuracy: 0.9874 - val_loss: 0.0480
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 83s 38ms/step - a